In [ ]:
%pip install numpy rasterio netCDF4 pyproj

In [ ]:
import os
import numpy as np
import rasterio
from rasterio.transform import from_origin
from rasterio.crs import CRS
from rasterio.warp import reproject, Resampling
from netCDF4 import Dataset
from pyproj import Transformer, Proj

# --- Config ---
INPUT_DIR = r"E:\GOES-R Lightning Data\Test Batch"
OUTPUT_DIR = r"E:\GOES-R Lightning Data\Raster Output"
ENERGY_THRESHOLD = 1.2e-05
PIXEL_SIZE = 1000  # 1 km
BUFFER_M = 161_000  # 100 miles in meters

# Florida approximate bounding box (lat/lon)
MIN_LON, MAX_LON = -87.63, -80.03
MIN_LAT, MAX_LAT = 24.52, 31.00

# EPSG codes
DST_CRS = CRS.from_epsg(5070)  # NAD83 / CONUS Albers

# --- Step 1: Define target raster ---
transformer = Transformer.from_crs("EPSG:4326", DST_CRS, always_xy=True)
min_x, min_y = transformer.transform(MIN_LON, MIN_LAT)
max_x, max_y = transformer.transform(MAX_LON, MAX_LAT)

# Add buffer
min_x -= BUFFER_M
min_y -= BUFFER_M
max_x += BUFFER_M
max_y += BUFFER_M

# Compute raster size
n_cols = int(np.ceil((max_x - min_x) / PIXEL_SIZE))
n_rows = int(np.ceil((max_y - min_y) / PIXEL_SIZE))

# Affine transform for target grid
target_transform = from_origin(min_x, max_y, PIXEL_SIZE, PIXEL_SIZE)

# Preallocate rasters
cumulative_energy = np.zeros((n_rows, n_cols), dtype=np.float64)
flash_count = np.zeros((n_rows, n_cols), dtype=np.int32)
max_energy = np.zeros((n_rows, n_cols), dtype=np.float64)
sum_for_mean = np.zeros((n_rows, n_cols), dtype=np.float64)
count_for_mean = np.zeros((n_rows, n_cols), dtype=np.int32)

# --- Step 2: Process NetCDF files ---
files = [os.path.join(INPUT_DIR, f) for f in os.listdir(INPUT_DIR) if f.endswith(".nc")]

for fpath in files:
    try:
        with Dataset(fpath, "r") as ds:
            slot = getattr(ds, "orbital_slot", "").strip().lower()
            if slot not in ["goes-east", "goes-west"]:
                continue

            proj_info = ds.variables["goes_imager_projection"]
            lon_origin = proj_info.longitude_of_projection_origin
            H = proj_info.perspective_point_height + proj_info.semi_major_axis
            r_eq = proj_info.semi_major_axis
            r_pol = proj_info.semi_minor_axis

            data = ds.variables["Total_Optical_energy"][:].astype(np.float64)
            fill_value = getattr(ds.variables["Total_Optical_energy"], "_FillValue", np.nan)
            data[data == fill_value] = np.nan
            data[data == 0] = np.nan  # skip zeros

            if np.all(np.isnan(data)):
                continue

            # --- Create source raster transform ---
            x = ds.variables["x"][:]
            y = ds.variables["y"][:]
            dx = (x[1] - x[0]) * H
            dy = (y[1] - y[0]) * H
            src_transform = from_origin(
                x[0] * H, y[0] * H, dx, dy
            )

            # --- Source CRS ---
            src_crs = CRS.from_proj4(
                f"+proj=geos +h={H} +lon_0={lon_origin} +a={r_eq} +b={r_pol} +sweep=x +units=m +no_defs"
            )


            # --- Reproject raster to target grid ---
            reprojected = np.zeros((n_rows, n_cols), dtype=np.float64)
            reproject(
                source=data,
                destination=reprojected,
                src_transform=src_transform,
                src_crs=src_crs,
                dst_transform=target_transform,
                dst_crs=DST_CRS,
                resampling=Resampling.bilinear,
                num_threads=-1
            )

            # Mask zeros / NaNs
            valid_mask = ~np.isnan(reprojected) & (reprojected > 0)

            # --- Update cumulative energy and max ---
            cumulative_energy[valid_mask] += reprojected[valid_mask]
            max_energy[valid_mask] = np.maximum(max_energy[valid_mask], reprojected[valid_mask])

            # --- Update flash count and mean ---
            thresh_mask = reprojected > ENERGY_THRESHOLD
            flash_count[thresh_mask] += 1
            sum_for_mean[thresh_mask] += reprojected[thresh_mask]
            count_for_mean[thresh_mask] += 1

    except Exception as e:
        print(f"Error processing {fpath}: {e}")
        continue

# --- Step 3: Compute mean and weighted intensity ---
mean_energy = sum_for_mean / np.maximum(count_for_mean, 1)
weighted_intensity = cumulative_energy * flash_count

# --- Step 4: Save rasters as GeoTIFFs (scaled for TIFF) ---
SCALE_FACTOR = 1e5  # scale up to preserve small values

def save_tiff(array, fname, scale=SCALE_FACTOR):
    path = os.path.join(OUTPUT_DIR, fname)
    with rasterio.open(
        path,
        'w',
        driver='GTiff',
        height=array.shape[0],
        width=array.shape[1],
        count=1,
        dtype='float32',  # keep float32 for precision
        crs=DST_CRS,
        transform=target_transform,
        compress='lzw'
    ) as dst:
        dst.write((array * scale).astype('float32'), 1)
    print(f"Saved {path} (scaled by {scale})")

save_tiff(cumulative_energy, "cumulative_energy.tif")
save_tiff(flash_count, "flash_count.tif")
save_tiff(max_energy, "max_energy.tif")
save_tiff(mean_energy, "mean_energy.tif")
save_tiff(weighted_intensity, "weighted_intensity.tif")

print("Processing complete!")


Saved E:\GOES-R Lightning Data\Raster Output\cumulative_energy.tif (scaled by 100000.0)
Saved E:\GOES-R Lightning Data\Raster Output\flash_count.tif (scaled by 100000.0)
Saved E:\GOES-R Lightning Data\Raster Output\max_energy.tif (scaled by 100000.0)
Saved E:\GOES-R Lightning Data\Raster Output\mean_energy.tif (scaled by 100000.0)
Saved E:\GOES-R Lightning Data\Raster Output\weighted_intensity.tif (scaled by 100000.0)
Processing complete!
